<a href="https://colab.research.google.com/github/aswinaus/graphrag/blob/main/Graph_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pyvis IPython cchardet datasets langchain==0.1.17 neo4j openai tiktoken langchain-community langchain-experimental json-repair

In [2]:
from getpass import getpass
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [3]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("aswinaus/Individual_income_tax_dataset_2019", download_mode="force_redownload")
df=pd.DataFrame(dataset['train'])

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

tax_statistics_dataset - Copy - Copy.csv:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19714 [00:00<?, ? examples/s]

In [4]:
df.head(10)

,STATEFIPS,STATE,zipcode,Size of adjusted gross income,No of returns,No of single returns,No of joint returns,No of head of household returns,Number of electronically filed returns,Number of computer prepared paper returns,...,Number of returns with net investment income tax,Net investment income tax amount,Number of returns with tax due at time of filing,Tax due at time of filing amount,Number of returns with total overpayments,Total overpayments amount,Number of returns with overpayments refunded,Overpayments refunded amount,Number of returns with credit to next years estimated tax,Credited to next years estimated tax amount
0,1,AL,0,"$1 under $25,000",778210,491030,84770,189600,712890,30670,...,0,0,62720,51936,671860,1700965,669570,1694792,1980,3512
1,1,AL,0,"$25,000 under $50,000",525940,247140,123910,139860,481760,18960,...,0,0,85860,122569,438020,1274802,435210,1266557,3670,7410
2,1,AL,0,"$50,000 under $75,000",285700,105140,128140,44560,260570,10670,...,0,0,73980,154932,212040,575315,208470,564202,5020,13653
3,1,AL,0,"$75,000 under $100,000",179070,38820,123110,13740,164300,5020,...,0,0,51330,139065,126850,401581,123310,388749,3040,10377
4,1,AL,0,"$100,000 under $200,000",257010,28180,216740,7150,236850,8400,...,90,141,104290,460071,152790,598248,144640,539385,9180,56257
5,1,AL,0,"$200,000 or more",74810,4540,66580,530,69330,1760,...,39430,141243,40030,946610,32060,681565,22420,257633,9450,372747
6,1,AL,35004,"$1 under $25,000",1450,990,180,250,1320,60,...,0,0,120,100,1230,2383,1220,2365,0,0
7,1,AL,35004,"$25,000 under $50,000",1370,700,280,340,1260,50,...,0,0,220,261,1150,2995,1140,2984,0,0
8,1,AL,35004,"$50,000 under $75,000",970,410,370,160,900,50,...,0,0,240,410,730,1814,720,1799,40,46
9,1,AL,35004,"$75,000 under $100,000",650,130,460,60,610,0,...,0,0,170,391,480,1666,470,1468,0,0


In [ ]:
# @title Knowledge Graph Builder & Visualizer
# @markdown Note that rendering the graph will take a minute or two under the default 10% sample size, longer with higher sample sizes.
sample_size = 0.66 # @param {type:"number", default:0.10}
import pandas as pd
import networkx as nx
from pyvis.network import Network
from IPython.display import IFrame
from IPython.display import Markdown, HTML


colors = {
    'HOTEL': 'lightblue',
    'REVIEW': 'orange',
    'LOCALITY': 'red',
    'COUNTRY': 'green',
    'TRIP_DATE': 'magenta',
    'SENTIMENT': 'purple',
}
sizes = {
    'HOTEL': 20,
    'REVIEW': 15,
    'LOCALITY': 25,
    'COUNTRY': 30,
    'TRIP_DATE': 10,
    'SENTIMENT': 15,
}

# Stratify based on 'rating_value', 'locality', 'trip_period', 'hotel_name' columns
stratify_cols = ['rating_value', 'locality', 'trip_period', 'hotel_name']
sample_size = 0.10

# Perform stratified sampling
sampled_df = df.groupby(stratify_cols, group_keys=False).apply(lambda x: x.sample(frac=sample_size))

# Initialize a directed graph
G = nx.DiGraph()

# Adding nodes with the entity type as a node attribute
for index, row in sampled_df.iterrows():
    if row['hotel_name'] not in G:
      G.add_node(row['hotel_name'],
                 entity='HOTEL',
                 description=row['hotel_description'],
                 url=row['hotel_url'],
                 image=row['hotel_image'],
                 address=row['street_address'],
                 price_range=row['price_range'],
                 locality=row['locality'],
                 color=colors.get('HOTEL', 'gray'),
                 size=sizes.get('HOTEL', 10), )
    if row['locality'] not in G:
      G.add_node(row['locality'], entity='LOCALITY', color=colors.get('LOCALITY', 'gray'),
                 size=sizes.get('LOCALITY', 10))

    if row['country'] not in G:
      G.add_node(row['country'], entity='COUNTRY', color=colors.get('COUNTRY', 'gray'),
                 size=sizes.get('COUNTRY', 10))

    if row['trip_period'] != None and row['trip_period'] not in G:
        G.add_node(row['trip_period'], entity='TRIP_DATE', color=colors['TRIP_DATE'], size=sizes['TRIP_DATE'])

    if not G.has_edge(row['hotel_name'], row['locality']):
      G.add_edge(row['hotel_name'], row['locality'], relationship='LOCATED_IN', color='red')

    if not G.has_edge(row['locality'], row['country']):
      G.add_edge(row['locality'], row['country'], relationship='LOCATED_IN', color='red')


    G.add_node(
               row['review_title'],
               entity='REVIEW',
               text=row['review_text'],
               tripdate=row['trip_period'],
               rating_value=row['rating_value'],
               color=colors.get('REVIEW', 'gray'),
               size=sizes.get('REVIEW', 10))
    # Adding edges to denote relationships
    G.add_edge(row['hotel_name'], row['review_title'], relationship='HAS_REVIEW')
    if row['trip_period'] != None:
      G.add_edge(row['review_title'], row['trip_period'], relationship='ON_DATE')

    # Add sentiment
    sentiment = 'Positive' if row['sentiment'] > 0 else 'Negative' if row['sentiment'] < 0 else 'Neutral'
    if sentiment not in G:
        G.add_node(sentiment, entity='SENTIMENT', color=colors.get('SENTIMENT', 'gray'), size=sizes.get('SENTIMENT', 10))
    G.add_edge(row['review_title'], sentiment, relationship='HAS_SENTIMENT')


# Step 4: Visualization (Optional)
# Convert to a pyvis network
nt = Network('700px', '700px', notebook=True, cdn_resources='in_line')
# if you are not in a Jupyter environment, you might need to set notebook=False
nt.from_nx(G)
nt.toggle_physics(True)  # Enable force-directed algorithm
nt.save_graph('hotel_reviews_graph.html')
nt.show('hotel_reviews_graph.html')

HTML('hotel_reviews_graph.html')